# ध्यान मेकानिज्म र ट्रान्सफर्मरहरू

पुनरावर्ती नेटवर्कहरूको एक प्रमुख कमजोरी यो हो कि अनुक्रमका सबै शब्दहरूले नतिजामा समान प्रभाव पार्छन्। यसले नामित इकाई पहिचान र मेसिन अनुवाद जस्ता अनुक्रम-देखि-अनुक्रम कार्यहरूको लागि मानक LSTM एन्कोडर-डिकोडर मोडेलहरूसँग उपयुक्त प्रदर्शन प्रदान गर्दैन। वास्तविकतामा, इनपुट अनुक्रमका केही विशेष शब्दहरूले अन्य शब्दहरूको तुलनामा अनुक्रमिक आउटपुटमा बढी प्रभाव पार्छन्।

मेसिन अनुवाद जस्ता अनुक्रम-देखि-अनुक्रम मोडेललाई विचार गर्नुहोस्। यो दुई पुनरावर्ती नेटवर्कहरूद्वारा कार्यान्वयन गरिन्छ, जहाँ एउटा नेटवर्क (**एन्कोडर**) इनपुट अनुक्रमलाई लुकेको अवस्थामा सङ्कुचित गर्छ, र अर्को नेटवर्क (**डिकोडर**) यस लुकेको अवस्थालाई अनुवादित नतिजामा विस्तार गर्छ। यस दृष्टिकोणको समस्या यो हो कि नेटवर्कको अन्तिम अवस्थाले वाक्यको सुरुवात सम्झन गाह्रो मान्छ, जसले गर्दा लामो वाक्यहरूमा मोडेलको गुणस्तर कमजोर हुन्छ।

**ध्यान मेकानिज्महरू** RNN को प्रत्येक आउटपुट भविष्यवाणीमा प्रत्येक इनपुट भेक्टरको सन्दर्भात्मक प्रभावलाई तौल दिने माध्यम प्रदान गर्छन्। यसलाई कार्यान्वयन गर्ने तरिका भनेको इनपुट RNN का मध्यवर्ती अवस्थाहरू र आउटपुट RNN बीच छोटो मार्गहरू सिर्जना गर्नु हो। यसरी, आउटपुट प्रतीक $y_t$ उत्पन्न गर्दा, हामी सबै इनपुट लुकेका अवस्थाहरू $h_i$ लाई विभिन्न तौल गुणांक $\alpha_{t,i}$ का साथ विचार गर्नेछौं।

![एन्कोडर/डिकोडर मोडेलमा एडिटिभ ध्यान तह देखाउने छवि](../../../../../translated_images/ne/encoder-decoder-attention.7a726296894fb567.webp)
*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) मा एडिटिभ ध्यान मेकानिज्म भएको एन्कोडर-डिकोडर मोडेल, [यो ब्लग पोस्ट](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html) बाट उद्धृत]*

ध्यान म्याट्रिक्स $\{\alpha_{i,j}\}$ ले इनपुटका निश्चित शब्दहरूले आउटपुट अनुक्रमको कुनै शब्द उत्पन्न गर्न कत्तिको भूमिका खेल्छन् भन्ने प्रतिनिधित्व गर्दछ। तल यस्तो म्याट्रिक्सको उदाहरण दिइएको छ:

![RNNsearch-50 द्वारा फेला परेको नमूना एलाइनमेन्ट देखाउने छवि, Bahdanau - arviz.org बाट लिइएको](../../../../../translated_images/ne/bahdanau-fig3.09ba2d37f202a6af.webp)

*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) बाट लिइएको चित्र (Fig.3)*

ध्यान मेकानिज्महरू प्राकृतिक भाषा प्रशोधनमा हालको वा निकट भविष्यको उत्कृष्ट अवस्थाको लागि जिम्मेवार छन्। ध्यान थप्दा मोडेल प्यारामिटरहरूको संख्या धेरै बढ्छ, जसले गर्दा RNN हरूसँग स्केलिङ समस्याहरू देखा पर्छ। RNN हरूको स्केलिङको प्रमुख बाधा भनेको मोडेलहरूको पुनरावर्ती प्रकृतिले प्रशिक्षणलाई ब्याच र समानान्तर बनाउन चुनौतीपूर्ण बनाउँछ। RNN मा अनुक्रमका प्रत्येक तत्वलाई क्रमिक रूपमा प्रशोधन गर्नुपर्छ, जसले गर्दा यसलाई सजिलै समानान्तर बनाउन सकिँदैन।

ध्यान मेकानिज्महरूको अपनत्व र यस बाधाले आज हामीले BERT देखि OpenGPT3 सम्म प्रयोग गर्ने उत्कृष्ट ट्रान्सफर्मर मोडेलहरूको सिर्जनालाई जन्म दियो।

## ट्रान्सफर्मर मोडेलहरू

प्रत्येक अघिल्लो भविष्यवाणीको सन्दर्भलाई अर्को मूल्याङ्कन चरणमा अगाडि बढाउने सट्टा, **ट्रान्सफर्मर मोडेलहरू** ले **पोजिसनल इन्कोडिङहरू** र **ध्यान** प्रयोग गरेर दिइएको इनपुटको सन्दर्भलाई प्रदान गरिएको पाठको झ्यालभित्र समेट्छन्। तलको छविले पोजिसनल इन्कोडिङहरू र ध्यानले कसरी झ्यालभित्रको सन्दर्भ समेट्न सक्छ भन्ने देखाउँछ।

![ट्रान्सफर्मर मोडेलहरूमा मूल्याङ्कनहरू कसरी गरिन्छ भन्ने देखाउने एनिमेटेड GIF।](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

किनभने प्रत्येक इनपुट स्थितिलाई स्वतन्त्र रूपमा प्रत्येक आउटपुट स्थितिमा म्याप गरिन्छ, ट्रान्सफर्मरहरूले RNN हरूको तुलनामा राम्रो समानान्तर गर्न सक्छन्, जसले गर्दा धेरै ठूलो र अझ अभिव्यक्तिपूर्ण भाषा मोडेलहरू सक्षम बनाउँछ। प्रत्येक ध्यान हेडले शब्दहरू बीचका विभिन्न सम्बन्धहरू सिक्न प्रयोग गर्न सकिन्छ, जसले प्राकृतिक भाषा प्रशोधनका कार्यहरूमा सुधार ल्याउँछ।

## साधारण ट्रान्सफर्मर मोडेल निर्माण

Keras मा बिल्ट-इन ट्रान्सफर्मर तह समावेश छैन, तर हामी आफैं बनाउन सक्छौं। पहिलेझैं, हामी AG News डेटासेटको पाठ वर्गीकरणमा केन्द्रित हुनेछौं, तर यो उल्लेख गर्न लायक छ कि ट्रान्सफर्मर मोडेलहरूले अझ कठिन NLP कार्यहरूमा उत्कृष्ट नतिजा देखाउँछन्।


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Keras मा नयाँ लेयरहरूले `Layer` वर्गलाई सबक्लास गर्नुपर्छ, र `call` मेथड कार्यान्वयन गर्नुपर्छ। सुरु गरौं **Positional Embedding** लेयरबाट। हामी [Keras को आधिकारिक डकुमेन्टेसनबाट केही कोड](https://keras.io/examples/nlp/text_classification_with_transformer/) प्रयोग गर्नेछौं। हामीले सबै इनपुट सिक्वेन्सहरूलाई `maxlen` लम्बाइमा प्याड गर्नेछौं भन्ने मान्नेछौं।


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

यो तह दुईवटा `Embedding` तहहरूबाट बनेको छ: टोकनहरूलाई एम्बेड गर्न (जसरी हामीले पहिले चर्चा गरिसकेका छौं) र टोकनको स्थानहरूलाई एम्बेड गर्न। टोकन स्थानहरू `tf.range` प्रयोग गरेर 0 देखि `maxlen` सम्मको प्राकृतिक संख्याहरूको क्रमको रूपमा सिर्जना गरिन्छ, र त्यसपछि एम्बेडिङ तहमा पठाइन्छ। दुईवटा परिणामस्वरूप प्राप्त एम्बेडिङ भेक्टरहरूलाई जोडिन्छ, जसले इनपुटको स्थानगत-एम्बेड गरिएको प्रतिनिधित्व उत्पादन गर्दछ, जसको आकार `maxlen`$\times$`embed_dim` हुन्छ।

अब, हामी ट्रान्सफर्मर ब्लक कार्यान्वयन गरौं। यसले पहिले परिभाषित गरिएको एम्बेडिङ तहको आउटपुटलाई लिनेछ:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

अब, हामी पूर्ण ट्रान्सफर्मर मोडेल परिभाषित गर्न तयार छौं:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT ट्रान्सफर्मर मोडेलहरू

**BERT** (Bidirectional Encoder Representations from Transformers) एक धेरै ठूलो बहु-स्तरीय ट्रान्सफर्मर नेटवर्क हो जसमा *BERT-base* का लागि १२ तहहरू र *BERT-large* का लागि २४ तहहरू छन्। यो मोडेललाई पहिलो चरणमा ठूलो पाठ डाटाको संग्रह (WikiPedia + किताबहरू) मा असुपरभाइज्ड तालिम (वाक्यमा लुकेका शब्दहरूको भविष्यवाणी गर्दै) प्रयोग गरेर प्रि-ट्रेन गरिन्छ। प्रि-ट्रेनिङको क्रममा मोडेलले भाषाको गहिरो समझ हासिल गर्छ, जसलाई अन्य डाटासेटहरूसँग फाइन ट्युनिङ गरेर उपयोग गर्न सकिन्छ। यस प्रक्रियालाई **ट्रान्सफर लर्निङ** भनिन्छ।

![picture from http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/ne/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

BERT, DistilBERT, BigBird, OpenGPT3 लगायतका ट्रान्सफर्मर आर्किटेक्चरका धेरै भेरिएसनहरू छन् जसलाई फाइन ट्युन गर्न सकिन्छ।

अब हामी प्रि-ट्रेन गरिएको BERT मोडेललाई हाम्रो परम्परागत क्रम वर्गीकरण समस्याको समाधानका लागि कसरी प्रयोग गर्न सकिन्छ भन्ने कुरा हेर्नेछौं। हामी [official documentation](https://www.tensorflow.org/text/tutorials/classify_text_with_bert) बाट केही विचार र कोड लिनेछौं।

प्रि-ट्रेन गरिएको मोडेलहरू लोड गर्नका लागि, हामी **Tensorflow hub** प्रयोग गर्नेछौं। पहिलो चरणमा, BERT-विशेष भेक्टराइजर लोड गरौं:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

यो महत्त्वपूर्ण छ कि तपाईले त्यही भेक्टराइजर प्रयोग गर्नुहोस् जुन मूल नेटवर्कलाई तालिम दिन प्रयोग गरिएको थियो। साथै, BERT भेक्टराइजरले तीनवटा कम्पोनेन्टहरू फिर्ता गर्छ:
* `input_word_ids`, जुन इनपुट वाक्यको लागि टोकन नम्बरहरूको क्रम हो
* `input_mask`, जसले क्रमको कुन भाग वास्तविक इनपुट हो र कुन भाग प्याडिङ हो भनेर देखाउँछ। यो `Masking` तहले उत्पादन गर्ने मास्कसँग मिल्दोजुल्दो छ
* `input_type_ids` भाषा मोडलिङ कार्यहरूको लागि प्रयोग गरिन्छ, र एउटै क्रममा दुईवटा इनपुट वाक्यहरू निर्दिष्ट गर्न अनुमति दिन्छ।

त्यसपछि, हामी BERT फिचर एक्स्ट्र्याक्टरलाई इनस्ट्यान्सियेट गर्न सक्छौं:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


त्यसैले, BERT लेयरले केही उपयोगी नतिजाहरू फर्काउँछ:
* `pooled_output` अनुक्रममा सबै टोकनहरूको औसत निकाल्ने परिणाम हो। यसलाई सम्पूर्ण नेटवर्कको बौद्धिक अर्थपूर्ण एम्बेडिङको रूपमा हेर्न सकिन्छ। यो हाम्रो अघिल्लो मोडेलमा `GlobalAveragePooling1D` लेयरको आउटपुटसँग समान छ।
* `sequence_output` अन्तिम ट्रान्सफर्मर लेयरको आउटपुट हो (यो माथिको मोडेलमा `TransformerBlock` को आउटपुटसँग मेल खान्छ)।
* `encoder_outputs` सबै ट्रान्सफर्मर लेयरहरूको आउटपुटहरू हुन्। किनकि हामीले 4-लेयर BERT मोडेल लोड गरेका छौं (जसको नाममा `4_H` समावेश छ भन्ने कुराबाट तपाईं अनुमान गर्न सक्नुहुन्छ), यसमा 4 टेन्सरहरू छन्। अन्तिम टेन्सर `sequence_output` जस्तै हो।

अब हामी अन्त्य-देखि-अन्त्य वर्गीकरण मोडेल परिभाषित गर्नेछौं। हामी *functional model definition* प्रयोग गर्नेछौं, जहाँ हामी मोडेल इनपुट परिभाषित गर्छौं, र त्यसपछि यसको आउटपुट गणना गर्न अभिव्यक्तिहरूको श्रृंखला प्रदान गर्छौं। हामी BERT मोडेलको वजनहरूलाई non-trainable बनाउनेछौं, र केवल अन्तिम वर्गीकरणकर्ता मात्र प्रशिक्षण गर्नेछौं:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


यद्यपि थोरै ट्रेन गर्न सकिने प्यारामिटरहरू छन्, प्रक्रिया निकै ढिलो छ, किनकि BERT फिचर एक्स्ट्र्याक्टर कम्प्युटेशनल रूपमा भारी छ। यस्तो देखिन्छ कि हामीले उचित शुद्धता प्राप्त गर्न सकेनौं, या त ट्रेनिङको कमीका कारण, या मोडेल प्यारामिटरहरूको अभावका कारण।

अब BERT को वजनहरू अनफ्रिज गरेर यसलाई पनि ट्रेन गर्ने प्रयास गरौं। यसका लागि धेरै सानो लर्निङ रेट आवश्यक छ, साथै **warmup** सहितको अधिक सावधानीपूर्वक ट्रेनिङ रणनीति चाहिन्छ, **AdamW** अप्टिमाइजर प्रयोग गरेर। हामी `tf-models-official` प्याकेज प्रयोग गरेर अप्टिमाइजर बनाउनेछौं:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

जसरी तपाईंले देख्न सक्नुहुन्छ, प्रशिक्षण प्रक्रिया अलि ढिलो हुन्छ - तर तपाईंले केही इपोक्स (५-१०) को लागि मोडेललाई प्रशिक्षण गरेर पहिले प्रयोग गरिएका तरिकाहरूसँग तुलना गर्दै उत्कृष्ट नतिजा प्राप्त गर्न प्रयास गर्न सक्नुहुन्छ।

## Huggingface Transformers पुस्तकालय

Transformer मोडेलहरू प्रयोग गर्ने अर्को धेरै सामान्य (र अलि सरल) तरिका [HuggingFace प्याकेज](https://github.com/huggingface/) हो, जसले विभिन्न NLP कार्यहरूको लागि सरल निर्माण ब्लकहरू प्रदान गर्दछ। यो Tensorflow र PyTorch दुबैका लागि उपलब्ध छ, जुन अर्को धेरै लोकप्रिय न्यूरल नेटवर्क फ्रेमवर्क हो।

> **Note**: यदि तपाईंलाई Transformers पुस्तकालय कसरी काम गर्छ भनेर हेर्नमा चासो छैन भने - तपाईं यो नोटबुकको अन्त्यसम्म जान सक्नुहुन्छ, किनभने यहाँ माथि गरिएका कुराहरूभन्दा धेरै फरक केही देखिने छैन। हामी BERT मोडेललाई फरक पुस्तकालय र धेरै ठूलो मोडेल प्रयोग गरेर प्रशिक्षण गर्ने उही चरणहरू दोहोर्याउनेछौं। त्यसैले, यो प्रक्रियामा केही लामो प्रशिक्षण समावेश हुन्छ, त्यसैले तपाईंले केवल कोड हेर्न चाहन सक्नुहुन्छ।

हेरौं, हाम्रो समस्या [Huggingface Transformers](http://huggingface.co) प्रयोग गरेर कसरी समाधान गर्न सकिन्छ।


पहिलो कुरा हामीले गर्ने भनेको प्रयोग गर्न चाहेको मोडेल चयन गर्नु हो। केही बिल्ट-इन मोडेलहरूका अतिरिक्त, Huggingface मा [अनलाइन मोडेल रिपोजिटरी](https://huggingface.co/models) उपलब्ध छ, जहाँ तपाईं समुदायद्वारा बनाइएका धेरै प्रि-ट्रेन गरिएको मोडेलहरू फेला पार्न सक्नुहुन्छ। ती सबै मोडेलहरू केवल मोडेलको नाम प्रदान गरेर लोड र प्रयोग गर्न सकिन्छ। मोडेलका लागि आवश्यक सबै बाइनरी फाइलहरू स्वचालित रूपमा डाउनलोड हुनेछन्।

कहिलेकाहीँ तपाईंलाई आफ्नै मोडेलहरू लोड गर्न आवश्यक पर्न सक्छ, यस्तो अवस्थामा तपाईंले सबै सम्बन्धित फाइलहरू समावेश भएको डाइरेक्टरी निर्दिष्ट गर्न सक्नुहुन्छ, जसमा टोकनाइजरका लागि प्यारामिटरहरू, `config.json` फाइलमा मोडेलका प्यारामिटरहरू, बाइनरी वेटहरू आदि समावेश हुन्छन्।

मोडेलको नामबाट, हामी मोडेल र टोकनाइजर दुवैलाई इनिसियलाइज गर्न सक्छौं। टोकनाइजरबाट सुरु गरौं:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` वस्तुमा `encode` कार्य हुन्छ जसलाई पाठलाई सिधै एन्कोड गर्न प्रयोग गर्न सकिन्छ:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

हामी टोकनाइजरलाई अनुक्रमलाई मोडेलमा पास गर्न उपयुक्त तरिकामा एन्कोड गर्न प्रयोग गर्न सक्छौं, जस्तै `token_ids`, `input_mask` क्षेत्रहरू समावेश गरेर। हामीले `return_tensors='tf'` तर्क प्रदान गरेर Tensorflow टेन्सरहरू चाहन्छौं भनेर पनि निर्दिष्ट गर्न सक्छौं:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

हाम्रो अवस्थामा, हामीले `bert-base-uncased` नामक प्रि-ट्रेन गरिएको BERT मोडेल प्रयोग गर्नेछौं। *Uncased* ले मोडेल केस-इन्सेन्सिटिभ (अक्षरको ठूला-साना फरक नमान्ने) भएको जनाउँछ।

मोडेललाई प्रशिक्षण गर्दा, हामीले टोकनाइज गरिएको अनुक्रम इनपुटको रूपमा प्रदान गर्नुपर्छ, त्यसैले हामी डाटा प्रशोधन पाइपलाइन डिजाइन गर्नेछौं। किनभने `tokenizer.encode` एउटा Python फङ्सन हो, हामी यसलाई अघिल्लो युनिटमा जस्तै `py_function` प्रयोग गरेर कल गर्ने समान तरिका अपनाउनेछौं:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

अब हामी `BertForSequenceClassification` प्याकेज प्रयोग गरेर वास्तविक मोडेल लोड गर्न सक्छौं। यसले सुनिश्चित गर्दछ कि हाम्रो मोडेलसँग वर्गीकरणको लागि आवश्यक आर्किटेक्चर पहिले नै छ, अन्तिम वर्गीकरणकर्ता सहित। तपाईंले चेतावनी सन्देश देख्नुहुनेछ जसले अन्तिम वर्गीकरणकर्ताका तौलहरू आरम्भ गरिएको छैन भनी जनाउँछ, र मोडेललाई पूर्व-प्रशिक्षण आवश्यक पर्छ - यो पूर्ण रूपमा ठीक छ, किनभने हामीले गर्न लागेको कुरा यही हो!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


जस्तो कि तपाईंले `summary()` बाट देख्न सक्नुहुन्छ, मोडेलमा लगभग ११० मिलियन प्यारामिटरहरू छन्! सम्भवतः, यदि हामी सानो डेटासेटमा साधारण वर्गीकरण कार्य चाहन्छौं भने, हामी BERT बेस लेयरलाई प्रशिक्षण गर्न चाहँदैनौं:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


अब हामी प्रशिक्षण सुरु गर्न तयार छौं!

> **Note**: पूर्ण-स्तरको BERT मोडेल प्रशिक्षण गर्न धेरै समय लाग्न सक्छ! त्यसैले हामी यसलाई पहिलो ३२ ब्याचहरूका लागि मात्र प्रशिक्षण गर्नेछौं। यो केवल मोडेल प्रशिक्षण कसरी सेटअप गरिन्छ भन्ने देखाउनको लागि हो। यदि तपाईं पूर्ण-स्तरको प्रशिक्षण प्रयास गर्न इच्छुक हुनुहुन्छ भने - `steps_per_epoch` र `validation_steps` प्यारामिटरहरू हटाउनुहोस्, र प्रतीक्षा गर्न तयार हुनुहोस्!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


यदि तपाईं iterations को संख्या बढाउनुहुन्छ, पर्याप्त समय पर्खनुहुन्छ, र धेरै epochs को लागि प्रशिक्षण गर्नुहुन्छ भने, तपाईं अपेक्षा गर्न सक्नुहुन्छ कि BERT वर्गीकरणले हामीलाई सबैभन्दा राम्रो accuracy दिन्छ! यसको कारण BERT ले पहिले नै भाषाको संरचना राम्रोसँग बुझिसकेको छ, र हामीलाई केवल अन्तिम classifier लाई fine-tune गर्न आवश्यक छ। तर, किनभने BERT ठूलो मोडेल हो, सम्पूर्ण प्रशिक्षण प्रक्रिया लामो समय लाग्छ, र गम्भीर computational शक्ति आवश्यक पर्छ! (GPU, र सम्भवतः एकभन्दा बढी GPU).

> **Note:** हाम्रो उदाहरणमा, हामीले सबैभन्दा सानो pre-trained BERT मोडेलमध्ये एक प्रयोग गरिरहेका छौं। ठूला मोडेलहरू छन् जसले सम्भवतः अझ राम्रो नतिजा दिन सक्छ।


## मुख्य कुरा

यस इकाईमा, हामीले **ट्रान्सफर्मरहरू** मा आधारित धेरै हालसालैका मोडल आर्किटेक्चरहरू हेरेका छौं। हामीले तिनीहरूलाई हाम्रो पाठ वर्गीकरण कार्यका लागि प्रयोग गरेका छौं, तर त्यस्तै गरी, BERT मोडलहरूलाई इकाई निकाल्ने, प्रश्न उत्तर दिने, र अन्य NLP कार्यहरूको लागि पनि प्रयोग गर्न सकिन्छ।

ट्रान्सफर्मर मोडलहरू NLP मा हालको अत्याधुनिक प्रविधि प्रतिनिधित्व गर्छन्, र प्रायः अवस्थामा, जब तपाईंले अनुकूल NLP समाधानहरू कार्यान्वयन गर्न थाल्नुहुन्छ, यो पहिलो समाधान हुनुपर्छ जससँग तपाईंले प्रयोग गर्न थाल्नुहुन्छ। तर, यदि तपाईं उन्नत न्यूरल मोडलहरू निर्माण गर्न चाहनुहुन्छ भने, यस मोड्युलमा छलफल गरिएका पुनरावर्ती न्यूरल नेटवर्कहरूको आधारभूत सिद्धान्तहरू बुझ्नु अत्यन्त महत्त्वपूर्ण छ।



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटि वा अशुद्धता हुन सक्छ। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
